In [1]:
!pip install pythainlp

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import warnings
import numpy as np
import csv
from random import shuffle

from pythainlp.tokenize import word_tokenize
from pythainlp.util import normalize
import pythainlp.word_vector as thword2vec
from pythainlp.corpus.common import thai_stopwords
from pythainlp import thai_punctuations
from pythainlp import word_vector

from keras.models import Model,load_model
from keras.layers import Input, Dense, GRU, Dropout,LSTM
from keras.utils import to_categorical
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix,classification_report
warnings.filterwarnings('ignore')


Using TensorFlow backend.


In [4]:
def Cleaning(sentences):
    # step 1: word tokenization 
    pre_words = [[w for w in word_tokenize(s, engine="newmm", keep_whitespace=False)] for s in sentences]

    # step 2: word normalization
    normalized_words = []
    for w in pre_words:
        temp_w = []
        for nw in w:
            temp_w.append(normalize(nw))
        normalized_words.append(temp_w)
        
    return normalized_words

In [5]:
#------------------------- Read data ------------------------------
#------- for traning ---------
fp = open("train_input.txt", "r", encoding = 'utf-8-sig')
X_data = [(line.strip()).split("::")[1] for line in fp]
fp = open("train_ans.txt", "r", encoding = 'utf-8-sig')
convert = {"H" : 0, "P" : 1, "M" : 2}
Y_data = [int(convert[(line.strip()).split("::")[1].strip()]) for line in fp]
fp.close
train_data = [i + "::" + str(j) for i, j in zip(X_data, Y_data)]
shuffle(train_data)
labels = [int(d.split("::")[1]) for d in train_data]
sentences = [d.split("::")[0] for d in train_data]
#------- for testing ---------
fp = open("input_test.txt", "r", encoding = 'utf-8-sig')
test_data = [(line.strip()).split("::")[1] for line in fp]
fp.close
#

<function TextIOWrapper.close()>

In [6]:
#------------------------- Preprocessing ------------------------------
#------- for traning ---------
token_sentences = Cleaning(sentences)
max_sentence_length = max([len(s) for s in token_sentences])
#max_sentence_length = 200
vocab = set([w for s in token_sentences for w in s])
vocab_size = len(vocab)
#------- for testing ---------
test_token_sentences = Cleaning(test_data)
text_max_sentence_length = max([len(s) for s in test_token_sentences])
#text_max_sentence_length = 200
test_vocab = set([w for s in test_token_sentences for w in s])
test_vocab_size = len(test_vocab)

max_sentence_length = max(max_sentence_length,text_max_sentence_length )

79


In [7]:
#------------------- Extract word vectors -------------------------
word_vector_length = 300
model = word_vector.get_model()
#------- for traning ---------
train_word_vectors = np.zeros((len(token_sentences),max_sentence_length,word_vector_length))
sample_count = 0
for s in token_sentences:
    word_count = 0
    for w in s:
        try:
            train_word_vectors[sample_count,max_sentence_length-word_count-1,:] = model[w]
            word_count = word_count+1
        except:
            pass
    sample_count = sample_count+1
#------- for testing ---------
test_word_vectors = np.zeros((len(test_token_sentences),max_sentence_length,word_vector_length))
sample_count = 0
for s in test_token_sentences:
    word_count = 0
    for w in s:
        try:
            test_word_vectors[sample_count,max_sentence_length-word_count-1,:] = model[w]
            word_count = word_count+1
        except:
            pass
    sample_count = sample_count+1


In [12]:
inputLayer = Input(shape=(max_sentence_length,word_vector_length,))
rnn1 = GRU(100, activation='relu')(inputLayer)
rnn1 = Dropout(0.5)(rnn1)
h1 = Dense(50, activation='sigmoid')(rnn1)
h2 = Dense(10, activation='softmax')(h1)
outputLayer = Dense(3, activation='sigmoid')(h2) #for 3 classes
model = Model(inputs=inputLayer, outputs=outputLayer)

#====================== Do not Edit ================================
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
#===================================================================
history = model.fit(train_word_vectors, to_categorical(labels), epochs=200, batch_size=32, validation_split = 0.2)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 79, 300)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 100)               120300    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 33        
Total params: 125,893
Trainable params: 125,893
Non-trainable params: 0
_____________________________________________________

2050/2050 [==============================] - 6s 3ms/step - loss: 0.0894 - accuracy: 0.9888 - val_loss: 0.3705 - val_accuracy: 0.9006
Epoch 50/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0882 - accuracy: 0.9888 - val_loss: 0.3980 - val_accuracy: 0.8947
Epoch 51/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0844 - accuracy: 0.9893 - val_loss: 0.3895 - val_accuracy: 0.8967
Epoch 52/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0830 - accuracy: 0.9888 - val_loss: 0.3725 - val_accuracy: 0.9064
Epoch 53/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0819 - accuracy: 0.9888 - val_loss: 0.3645 - val_accuracy: 0.9084
Epoch 54/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0811 - accuracy: 0.9888 - val_loss: 0.3566 - val_accuracy: 0.9064
Epoch 55/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0848 - accuracy: 0.9868 - val_loss: 0.4414 - val_accuracy

2050/2050 [==============================] - 6s 3ms/step - loss: 0.0336 - accuracy: 0.9956 - val_loss: 0.4718 - val_accuracy: 0.9006 0.0359 - accu
Epoch 105/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0445 - accuracy: 0.9922 - val_loss: 0.4792 - val_accuracy: 0.8967
Epoch 106/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0357 - accuracy: 0.9946 - val_loss: 0.4728 - val_accuracy: 0.9025
Epoch 107/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0347 - accuracy: 0.9951 - val_loss: 0.4764 - val_accuracy: 0.9006
Epoch 108/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0335 - accuracy: 0.9951 - val_loss: 0.4844 - val_accuracy: 0.9006
Epoch 109/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0328 - accuracy: 0.9956 - val_loss: 0.4875 - val_accuracy: 0.9006
Epoch 110/200
2050/2050 [==============================] - 7s 3ms/step - loss: 0.0348 - accuracy: 0.9946 - val_loss: 0

Epoch 160/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0237 - accuracy: 0.9961 - val_loss: 0.5348 - val_accuracy: 0.9006
Epoch 161/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0251 - accuracy: 0.9961 - val_loss: 0.5327 - val_accuracy: 0.9006
Epoch 162/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0234 - accuracy: 0.9961 - val_loss: 0.5535 - val_accuracy: 0.8986
Epoch 163/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0237 - accuracy: 0.9956 - val_loss: 0.5402 - val_accuracy: 0.8986
Epoch 164/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0292 - accuracy: 0.9951 - val_loss: 0.5337 - val_accuracy: 0.9045
Epoch 165/200
2050/2050 [==============================] - 7s 3ms/step - loss: 0.0258 - accuracy: 0.9956 - val_loss: 0.5263 - val_accuracy: 0.9025
Epoch 166/200
2050/2050 [==============================] - 6s 3ms/step - loss: 0.0272 - accuracy: 0.9951 - val_loss: 0

In [13]:
model.save('600610752_RE.h5')